In [8]:
import re
from transformers import DonutProcessor, VisionEncoderDecoderModel
from datasets import load_dataset
import torch
from PIL import Image

In [3]:
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")
model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base-finetuned-docvqa")

config.json:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/803M [00:00<?, ?B/s]

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [5]:
x_dict = {
    "219gbqQt+ML.jpg": "height",
    "218vf17tHkL.jpg": "weight",
    "21-VzxP3BDL.jpg": "item_volume",
    "217V+UhIrHL.jpg": "length",
    "11j0F4QOiFL.jpg": "height",
    "211sXYcOHcL.jpg": "height",
    "218zo3iJ2IL.jpg": "length",
    "213VIsNlvzL.jpg": "height",
    "21+quvMwZSL.jpg": "weight",
    "217+y-mckBL.jpg": "weight",
    "211EIgVhPEL.jpg": "voltage",
    "218tBdpDGPS.jpg": "length",
    "21-V2Kx5BVL.jpg": "length"
}

y_dict = {
    "219gbqQt+ML.jpg": "12 cm",
    "218vf17tHkL.jpg": "250 mg",
    "21-VzxP3BDL.jpg": "200 ml",
    "217V+UhIrHL.jpg": "5 cm",
    "11j0F4QOiFL.jpg": "2.75 inches",
    "211sXYcOHcL.jpg": "8 cm",
    "218zo3iJ2IL.jpg": "44.2 cm",
    "213VIsNlvzL.jpg": "11 cm",
    "21+quvMwZSL.jpg": "1.6 lbs",
    "217+y-mckBL.jpg": "400 mg",
    "211EIgVhPEL.jpg": "3.7 V",
    "218tBdpDGPS.jpg": "104.5 inches",
    "21-V2Kx5BVL.jpg": "80 inches"
}

In [10]:
image = Image.open(r'C:\Users\salos\OneDrive\Desktop\AmazonML-Hackathon\images\41-NCxNuBxL.jpg')

In [13]:
task_prompt = "<s_docvqa><s_question>{user_input}</s_question><s_answer>"
question = "What is the  of object:"
prompt = task_prompt.replace("{user_input}", question)
decoder_input_ids = processor.tokenizer(prompt, add_special_tokens=False, return_tensors="pt").input_ids
pixel_values = processor(image, return_tensors="pt").pixel_values

In [14]:
outputs = model.generate(
    pixel_values.to(device),
    decoder_input_ids=decoder_input_ids.to(device),
    max_length=model.decoder.config.max_position_embeddings,
    pad_token_id=processor.tokenizer.pad_token_id,
    eos_token_id=processor.tokenizer.eos_token_id,
    use_cache=True,
    bad_words_ids=[[processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,)

sequence = processor.batch_decode(outputs.sequences)[0]
sequence = sequence.replace(processor.tokenizer.eos_token, "").replace(processor.tokenizer.pad_token, "")
sequence = re.sub(r"<.*?>", "", sequence, count=1).strip()
print(processor.token2json(sequence))

{'question': 'What is the width of object:', 'answer': '20cm'}
